# 📚 Littérature Visuelle avec GenAI

**Module :** Examples Sectoriels - Littérature  
**Niveau :** 🟡 Intermédiaire  
**Technologies :** DALL-E 3, GPT-5 Vision  
**Durée estimée :** 30 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Illustrer des passages d'œuvres classiques
- [ ] Créer des character designs de personnages littéraires
- [ ] Visualiser des métaphores et symboles
- [ ] Concevoir des storyboards pour adaptations
- [ ] Enrichir l'analyse littéraire par l'image

## 📚 Prérequis

- Notebooks Foundation (01-1, 01-2) complétés
- Connaissance des œuvres littéraires étudiées

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

notebook_mode = "interactive"
skip_widgets = False
debug_level = "INFO"

# Configuration contenu
literary_period = "all"  # "classical", "romantic", "modern", "contemporary", "all"
genre = "all"  # "poetry", "novel", "theater", "all"
image_style = "artistic"  # "artistic", "realistic", "illustrated"
image_quality = "hd"

# Sujets littéraires
classical_works = ["odyssey", "divine_comedy", "don_quixote"]
french_classics = ["les_miserables", "le_petit_prince", "germinal"]

save_illustrations = True
export_study_pack = True

In [ ]:
# Setup environnement
import os
import sys
import json
import requests
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
from io import BytesIO
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import logging

GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'literature_visual'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('literature_visual')

openrouter_key = os.getenv('OPENROUTER_API_KEY')
if not openrouter_key:
    raise ValueError("❌ OPENROUTER_API_KEY manquante")

api_base_url = "https://openrouter.ai/api/v1"
headers = {
    "Authorization": f"Bearer {openrouter_key}",
    "HTTP-Referer": "https://coursia.myia.io",
    "X-Title": "CoursIA Literature Visual",
    "Content-Type": "application/json"
}

print(f"📚 Littérature Visuelle avec GenAI")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Période: {literary_period}, Genre: {genre}")
print(f"✅ Environnement configuré")

## 🎨 Section 1: Illustrations d'Œuvres Classiques

Visualisation de scènes clés de la littérature classique.

In [ ]:
# Templates illustrations classiques
classical_illustrations = {
    "les_miserables_barricade": {
        "title": "⚔️ Les Misérables - La Barricade",
        "prompt": """Create an artistic illustration of the barricade scene from Victor Hugo's Les Misérables:
- Street barricade made of furniture, paving stones, and debris
- Young revolutionaries (Gavroche, Enjolras, Marius) with red flags
- 19th century Paris architecture in background
- Dramatic lighting, tension in the air
- Revolutionary fervor and sacrifice
- Romantic style painting
- Rich colors: reds, grays, dramatic shadows

Style: 19th century Romantic painting. Epic, emotional, heroic.
Delacroix-inspired. Dramatic composition. High artistic quality.""",
        "author": "Victor Hugo",
        "work": "Les Misérables (1862)",
        "theme": "Révolution, sacrifice, justice sociale",
        "grade_level": "4ème/3ème"
    },
    "le_petit_prince_rose": {
        "title": "🌹 Le Petit Prince et sa Rose",
        "prompt": """Create a tender illustration from Le Petit Prince by Antoine de Saint-Exupéry:
- The Little Prince (small boy with golden hair and green suit)
- His beloved rose under a glass dome
- On asteroid B-612 (small spherical planet)
- Stars in the background
- Warm, gentle colors
- Whimsical, dreamlike atmosphere
- Watercolor painting style

Style: Delicate watercolor illustration. Gentle, poetic, melancholic.
Saint-Exupéry's original style inspiration. Soft pastels. Touching and beautiful.""",
        "author": "Antoine de Saint-Exupéry",
        "work": "Le Petit Prince (1943)",
        "theme": "Amour, responsabilité, essentialité",
        "grade_level": "CM2/6ème"
    },
    "germinal_mine": {
        "title": "⛏️ Germinal - La Mine",
        "prompt": """Create a powerful illustration of the mine scene from Émile Zola's Germinal:
- Dark coal mine interior with low ceilings
- Miners (men, women, children) working in harsh conditions
- Coal dust in the air, dim lantern light
- Exhaustion and hardship visible on faces
- Industrial Revolution era (1860s)
- Realistic, gritty depiction
- Dark palette: blacks, grays, browns, touches of lantern orange

Style: Social realism painting. Naturalist style like Zola's writing.
Dark, oppressive, but dignified. Strong social commentary. High artistic quality.""",
        "author": "Émile Zola",
        "work": "Germinal (1885)",
        "theme": "Condition ouvrière, lutte sociale, capitalisme",
        "grade_level": "3ème/Seconde"
    },
    "odyssey_sirens": {
        "title": "🌊 L'Odyssée - Les Sirènes",
        "prompt": """Create a classical illustration of Odysseus and the Sirens from Homer's Odyssey:
- Greek ship with Odysseus tied to the mast
- Beautiful but dangerous sirens on rocky islands
- Sailors with wax in their ears rowing
- Turbulent Mediterranean sea
- Ancient Greek ship design (wooden, sail)
- Mythological atmosphere
- Classical Greek art influence

Style: Classical Greek pottery art meets Renaissance painting.
Epic, mythological. Blues of sea, warm skin tones. Heroic composition.""",
        "author": "Homère",
        "work": "L'Odyssée (VIIIe siècle av. J-C)",
        "theme": "Tentation, ruse, voyage héroïque",
        "grade_level": "6ème/5ème"
    }
}

print("\n🎨 ILLUSTRATIONS D'ŒUVRES CLASSIQUES")
print("=" * 50)
for key, illus in classical_illustrations.items():
    print(f"\n{illus['title']}")
    print(f"   ✍️  {illus['author']} - {illus['work']}")
    print(f"   🎭 Thème: {illus['theme']}")
    print(f"   🎓 Niveau: {illus['grade_level']}")

In [ ]:
# Fonction génération illustrations
def generate_literary_illustration(prompt: str,
                                  title: str,
                                  category: str,
                                  quality: str = "hd") -> Dict[str, Any]:
    """Génère une illustration littéraire."""
    
    payload = {
        "model": "openai/dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1024x1792",  # Format portrait pour illustrations
        "quality": quality,
        "style": "vivid",  # Style vivid pour art
        "response_format": "url"
    }
    
    try:
        print(f"\n🎨 Génération: {title}")
        response = requests.post(
            f"{api_base_url}/images/generations",
            headers=headers,
            json=payload,
            timeout=90
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ Généré en {response.elapsed.total_seconds():.1f}s")
            
            return {
                "success": True,
                "image_url": result["data"][0]["url"],
                "title": title,
                "category": category,
                "revised_prompt": result["data"][0].get("revised_prompt", prompt),
                "timestamp": datetime.now().isoformat()
            }
        else:
            return {"success": False, "error": response.json().get("error", {}).get("message")}
    except Exception as e:
        return {"success": False, "error": str(e)}

def display_and_save(result: Dict[str, Any], metadata: Dict[str, str] = None) -> Optional[Image.Image]:
    """Affiche et sauvegarde avec métadonnées littéraires."""
    if not result.get('success'):
        print(f"   ❌ Échec: {result.get('error')}")
        return None
    
    try:
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            plt.figure(figsize=(10, 14))
            plt.imshow(image)
            
            title_text = result['title']
            if metadata:
                title_text += f"\n{metadata['author']} - {metadata['work']}"
            
            plt.title(title_text, fontsize=12, fontweight='bold', pad=20)
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            
            if save_illustrations:
                cat_dir = OUTPUT_DIR / result['category']
                cat_dir.mkdir(exist_ok=True)
                
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                safe_title = result['title'].replace(' ', '_')[:50]
                filename = f"{safe_title}_{timestamp}.png"
                
                image.save(cat_dir / filename)
                
                # Métadonnées littéraires
                if metadata:
                    meta_full = {**result, **metadata}
                    meta_file = cat_dir / f"{safe_title}_{timestamp}_meta.json"
                    with open(meta_file, 'w', encoding='utf-8') as f:
                        json.dump(meta_full, f, indent=2, ensure_ascii=False)
                
                print(f"   💾 Sauvegardé: {filename}")
            
            return image
    except Exception as e:
        print(f"   ❌ Erreur: {str(e)[:100]}")
    
    return None

# Génération: Les Misérables
print("\n" + "=" * 60)
print("⚔️ GÉNÉRATION: LES MISÉRABLES - LA BARRICADE")
print("=" * 60)

les_mis = classical_illustrations["les_miserables_barricade"]
les_mis_result = generate_literary_illustration(
    prompt=les_mis["prompt"],
    title=les_mis["title"],
    category="classical_illustrations",
    quality=image_quality
)

if les_mis_result['success']:
    les_mis_image = display_and_save(les_mis_result, {
        "author": les_mis["author"],
        "work": les_mis["work"],
        "theme": les_mis["theme"],
        "grade_level": les_mis["grade_level"]
    })
    
    print(f"\n💡 Usages pédagogiques:")
    print(f"   • Cours de français {les_mis['grade_level']}")
    print(f"   • Analyse scène de la barricade")
    print(f"   • Étude du romantisme")
    print(f"   • Thématique révolution et justice sociale")
    print(f"   • Support pour dissertation/exposé")

## 👤 Section 2: Character Design Littéraire

Création de designs visuels pour personnages emblématiques.

In [ ]:
# Templates character design
character_designs = {
    "jean_valjean": {
        "title": "👤 Jean Valjean (Les Misérables)",
        "prompt": """Create a character design sheet for Jean Valjean from Les Misérables:

Central portrait:
- Strong, weathered man in his 50s
- Kind but haunted eyes
- Gray hair, dignified bearing
- Simple but clean 19th century worker's clothing
- Compassionate expression showing inner nobility

Character sheet elements:
- Front view (main portrait)
- Side profile sketch
- Expression studies (stern, compassionate, determined)
- Age progression (young convict → older mayor → elderly)
- Symbolic items: bread (redemption), candlesticks (grace)

Style: Professional character design sheet. Realistic but artistic.
Pencil sketch aesthetic with color accents. Annotated design.""",
        "character": "Jean Valjean",
        "work": "Les Misérables",
        "traits": "Rédemption, bonté, justice"
    },
    "le_petit_prince": {
        "title": "👦 Le Petit Prince",
        "prompt": """Create a character design for Le Petit Prince:

Main character:
- Small boy, around 7-8 years old
- Golden blonde hair, slightly messy
- Green coat with golden buttons
- Yellow scarf flowing in wind
- Innocent, wise, melancholic expression
- Standing on small round planet (asteroid)

Design elements:
- Full body view
- Close-up of face showing wonder
- With his rose (under glass dome)
- With the fox (friendship moment)
- Looking at stars (contemplative)

Style: Watercolor character sheet. Gentle, poetic. Saint-Exupéry-inspired.
Soft colors. Whimsical but touching. Children's book illustration quality.""",
        "character": "Le Petit Prince",
        "work": "Le Petit Prince",
        "traits": "Innocence, sagesse, curiosité"
    },
    "cyrano": {
        "title": "🎭 Cyrano de Bergerac",
        "prompt": """Create a character design for Cyrano de Bergerac:

Main portrait:
- Musketeer with distinctive large nose
- Proud, noble bearing despite physical feature
- 17th century French musketeer costume
- Feathered hat, cape, sword
- Passionate, poetic expression
- Both warrior strength and poet's sensitivity visible

Design elements:
- Heroic stance with sword
- Writing poetry (with quill)
- Dueling pose (combining sword and verse)
- Profile emphasizing nose (his distinguishing feature)
- Expression showing inner pain vs outer bravado

Style: Theatrical character design. Swashbuckling romance.
Dramatic lighting and poses. 17th century costume accuracy.
Professional theater/film character concept quality.""",
        "character": "Cyrano de Bergerac",
        "work": "Cyrano de Bergerac",
        "traits": "Panache, amour non partagé, éloquence"
    }
}

print("\n👤 CHARACTER DESIGN LITTÉRAIRE")
print("=" * 50)
for key, char in character_designs.items():
    print(f"\n{char['title']}")
    print(f"   📖 Œuvre: {char['work']}")
    print(f"   🎭 Traits: {char['traits']}")

# Génération: Le Petit Prince
print("\n" + "=" * 60)
print("👦 GÉNÉRATION: LE PETIT PRINCE")
print("=" * 60)

prince = character_designs["le_petit_prince"]
prince_result = generate_literary_illustration(
    prompt=prince["prompt"],
    title=prince["title"],
    category="character_designs",
    quality=image_quality
)

if prince_result['success']:
    prince_image = display_and_save(prince_result, {
        "author": "Antoine de Saint-Exupéry",
        "work": prince["work"],
        "character": prince["character"],
        "traits": prince["traits"]
    })
    
    print(f"\n💡 Applications:")
    print(f"   • Cours de français CM2/6ème")
    print(f"   • Portrait de personnage")
    print(f"   • Adaptation théâtrale scolaire")
    print(f"   • Analyse personnage et symboles")

## 🎨 Section 3: Visualisation de Métaphores

Traduction visuelle de métaphores et symboles littéraires.

In [ ]:
# Templates métaphores visuelles
metaphor_visuals = {
    "baudelaire_albatross": {
        "title": "🦅 L'Albatros (Baudelaire) - Métaphore du Poète",
        "prompt": """Create a symbolic illustration for Baudelaire's poem 'L'Albatros':

Dual composition showing metaphor:
Upper half:
- Majestic albatross soaring gracefully in vast blue sky
- Wings spread wide, powerful, free
- Among clouds, sunlight, infinite space

Lower half:
- Same albatross on ship deck, awkward, trapped
- Sailors mocking, wings dragging on deck
- Confined, humiliated, out of its element

Center: Transformation/contrast line

Subtle overlay: Poet's silhouette suggesting parallel

Style: Symbolic, artistic. Split composition showing duality.
Dramatic contrast between freedom and constraint.
Romantic period painting style. Melancholic beauty.""",
        "poet": "Charles Baudelaire",
        "poem": "L'Albatros (Les Fleurs du Mal)",
        "metaphor": "Le poète incompris dans la société",
        "grade_level": "Première"
    },
    "hugo_light_shadow": {
        "title": "💡 Lumière et Ombre (Hugo) - Dualité Humaine",
        "prompt": """Create a symbolic illustration of Victor Hugo's light/shadow duality:

Composition:
- Human figure at center (Everyman)
- Left side bathed in golden light (redemption, good)
  * Angels, hope, rising sun, open sky
- Right side in dark shadows (fall, evil)
  * Demons, despair, darkness, closed spaces
- Figure standing at the crossroads
- Light and shadow battling across the figure
- Choice and free will emphasized

Style: Romantic symbolism. Dramatic chiaroscuro (light/dark contrast).
Caravaggio-inspired lighting. Philosophical, profound.
High artistic quality showing eternal human struggle.""",
        "author": "Victor Hugo",
        "theme": "Dualité bien/mal, rédemption",
        "metaphor": "Choix moral et libre arbitre",
        "grade_level": "Seconde/Première"
    },
    "rimbaud_drunken_boat": {
        "title": "🚤 Le Bateau Ivre (Rimbaud) - Liberté Poétique",
        "prompt": """Create a surreal illustration for Rimbaud's 'Le Bateau ivre':

Surreal scene:
- Small boat without captain or crew
- Floating on impossible, psychedelic waters
- Water colors: greens, purples, electric blues (synesthesia)
- Exotic visions: flying fish, floating islands, bizarre creatures
- Fragmented reality, dreamlike logic
- Sense of intoxicated freedom and danger
- Swirling, dynamic composition

Style: Surrealism meets Symbolism. Vivid, hallucinatory colors.
Dalí/Magritte-inspired impossible imagery. Wild, uncontrolled.
Youthful rebellion and poetic vision. High artistic quality.""",
        "poet": "Arthur Rimbaud",
        "poem": "Le Bateau ivre",
        "metaphor": "Liberté absolue, voyage intérieur, voyance poétique",
        "grade_level": "Première/Terminale"
    }
}

print("\n🎨 VISUALISATION DE MÉTAPHORES")
print("=" * 50)
for key, meta in metaphor_visuals.items():
    print(f"\n{meta['title']}")
    print(f"   📝 Métaphore: {meta['metaphor']}")
    print(f"   🎓 Niveau: {meta['grade_level']}")

# Génération: L'Albatros
print("\n" + "=" * 60)
print("🦅 GÉNÉRATION: L'ALBATROS (BAUDELAIRE)")
print("=" * 60)

albatross = metaphor_visuals["baudelaire_albatross"]
albatross_result = generate_literary_illustration(
    prompt=albatross["prompt"],
    title=albatross["title"],
    category="metaphors",
    quality=image_quality
)

if albatross_result['success']:
    albatross_image = display_and_save(albatross_result, {
        "author": albatross["poet"],
        "work": albatross["poem"],
        "metaphor": albatross["metaphor"],
        "grade_level": albatross["grade_level"]
    })
    
    print(f"\n💡 Utilisations:")
    print(f"   • Cours de français {albatross['grade_level']}")
    print(f"   • Analyse du symbolisme baudelairien")
    print(f"   • Commentaire composé sur L'Albatros")
    print(f"   • Étude métaphore poète/société")
    print(f"   • Oral du bac français")

## 🎬 Section 4: Storyboard pour Adaptation

Création de planches de storyboard pour adaptations théâtrales ou cinéma.

In [ ]:
# Template storyboard
storyboard_prompt = """Create a 6-panel storyboard for adapting 'Le Petit Prince' fox taming scene:

Panel 1: Little Prince arrives at Earth, alone
- Small figure on barren planet surface
- Wide shot showing isolation

Panel 2: First encounter with Fox
- Fox hiding behind wheat field
- Little Prince curious, approaching

Panel 3: Fox explains taming
- Medium shot, Fox and Prince talking
- Wheat mentioned (golden like Prince's hair)

Panel 4: The taming ritual begins
- Prince sitting, Fox approaching carefully
- Distance gradually closing over time

Panel 5: Friendship established
- Prince and Fox side by side
- Wheat field golden in background
- Connection visible

Panel 6: Fox's wisdom - 'On ne voit bien qu'avec le cœur'
- Close-up of Fox's wise expression
- Prince listening, understanding
- Emotional moment

Style: Clean storyboard panels. Professional film/animation pre-production.
Simple but clear compositions. Annotations for camera angles and movements.
Emotional progression clear across panels."""

print("\n🎬 STORYBOARD ADAPTATION")
print("=" * 50)
print("\n📋 Génération: Le Petit Prince - Scène de l'Apprivoisement")

storyboard_result = generate_literary_illustration(
    prompt=storyboard_prompt,
    title="🎬 Storyboard - Le Petit Prince et le Renard",
    category="storyboards",
    quality=image_quality
)

if storyboard_result['success']:
    storyboard_image = display_and_save(storyboard_result, {
        "author": "Antoine de Saint-Exupéry",
        "work": "Le Petit Prince",
        "scene": "L'apprivoisement du renard",
        "purpose": "Adaptation théâtrale/cinéma scolaire"
    })
    
    print(f"\n✅ Storyboard généré")
    print(f"\n💡 Applications pédagogiques:")
    print(f"   • Projet d'adaptation théâtrale")
    print(f"   • Cours de français (séquence théâtre)")
    print(f"   • Atelier cinéma/audiovisuel")
    print(f"   • Analyse mise en scène")
    print(f"   • Travail de groupe créatif")
    print(f"   • Compréhension narrative visuelle")

## 📦 Section 5: Pack d'Étude Littéraire

Export complet pour enseignants de français.

In [ ]:
if export_study_pack:
    print("\n📦 EXPORT PACK D'ÉTUDE LITTÉRAIRE")
    print("=" * 50)
    
    pack_dir = OUTPUT_DIR / f"literature_pack_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    pack_dir.mkdir(exist_ok=True)
    
    (pack_dir / 'illustrations').mkdir(exist_ok=True)
    (pack_dir / 'characters').mkdir(exist_ok=True)
    (pack_dir / 'metaphors').mkdir(exist_ok=True)
    (pack_dir / 'storyboards').mkdir(exist_ok=True)
    (pack_dir / 'study_guides').mkdir(exist_ok=True)
    
    print(f"\n📁 Structure créée: {pack_dir.name}")
    
    # Collecte des ressources
    resources = []
    if 'les_mis_result' in locals() and les_mis_result.get('success'):
        resources.append(('les_miserables', les_mis_result, classical_illustrations['les_miserables_barricade']))
    if 'prince_result' in locals() and prince_result.get('success'):
        resources.append(('petit_prince_char', prince_result, character_designs['le_petit_prince']))
    if 'albatross_result' in locals() and albatross_result.get('success'):
        resources.append(('albatros', albatross_result, metaphor_visuals['baudelaire_albatross']))
    if 'storyboard_result' in locals() and storyboard_result.get('success'):
        resources.append(('storyboard_pp', storyboard_result, {}))
    
    print(f"\n📊 {len(resources)} ressources à exporter")
    
    pack_index = {
        "title": "Pack d'Étude Littéraire Visuel",
        "created": datetime.now().isoformat(),
        "period": literary_period,
        "genre": genre,
        "resources": []
    }
    
    for key, result, template in resources:
        img_response = requests.get(result['image_url'], timeout=30)
        if img_response.status_code == 200:
            image = Image.open(BytesIO(img_response.content))
            
            subfolder = result['category']
            img_file = pack_dir / subfolder / f"{key}.png"
            image.save(img_file)
            
            # Guide d'étude associé
            if template:
                study_guide = f"""# {result['title']}

## 📖 Informations
- **Auteur:** {template.get('author', template.get('poet', 'N/A'))}
- **Œuvre:** {template.get('work', template.get('poem', 'N/A'))}
- **Niveau:** {template.get('grade_level', 'N/A')}

## 🎯 Objectifs Pédagogiques
{template.get('theme', template.get('metaphor', template.get('traits', 'N/A')))}

## 💡 Pistes d'Exploitation
1. Analyse visuelle de l'illustration
2. Comparaison texte/image
3. Discussion sur les choix artistiques
4. Création d'illustrations complémentaires
5. Rédaction inspirée par l'image

## 📝 Questions pour les Élèves
- Quels éléments visuels correspondent au texte?
- Quelles émotions l'image transmet-elle?
- Comment l'artiste interprète-t-il le passage/personnage/métaphore?
"""
                guide_file = pack_dir / 'study_guides' / f"{key}_guide.md"
                with open(guide_file, 'w', encoding='utf-8') as f:
                    f.write(study_guide)
            
            pack_index['resources'].append({
                "id": key,
                "title": result['title'],
                "category": result['category'],
                "file": f"{subfolder}/{key}.png",
                "guide": f"study_guides/{key}_guide.md" if template else None
            })
            
            print(f"   ✅ {result['title']}")
    
    # README principal
    readme = f"""# Pack d'Étude Littéraire Visuel

**Créé:** {datetime.now().strftime('%Y-%m-%d')}
**Période:** {literary_period}
**Genre:** {genre}

## 📚 Contenu du Pack

### Illustrations Classiques
Scènes clés d'œuvres majeures de la littérature française.

### Character Designs
Conceptions visuelles de personnages emblématiques.

### Métaphores Visuelles
Traductions visuelles de symboles et métaphores littéraires.

### Storyboards
Planches pour adaptations théâtrales ou cinématographiques.

## 🎓 Utilisation Pédagogique

Ces ressources sont conçues pour enrichir l'enseignement du français:
- Support visuel pour l'analyse littéraire
- Base de discussion en classe
- Inspiration pour travaux créatifs
- Comparaison texte/image
- Projets d'adaptation

## 📖 Niveaux
Collège (6ème à 3ème) et Lycée (Seconde à Terminale)
"""
    
    with open(pack_dir / 'README.md', 'w', encoding='utf-8') as f:
        f.write(readme)
    
    with open(pack_dir / 'pack_index.json', 'w', encoding='utf-8') as f:
        json.dump(pack_index, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ Pack exporté: {pack_dir.name}")
    print(f"📁 {len(pack_index['resources'])} ressources + guides d'étude")
else:
    print("\n⏭️  Export désactivé")

print("\n" + "=" * 50)
print("✅ Notebook Littérature Visuelle terminé!")
print("📚 Ressources pour enrichir l'enseignement littéraire")
print("=" * 50)